In [134]:
import weakref, sys, math
    
class GanttBase():
    def __init__(self, name):
        self.subitems = {}
        self.name = name
        
    def add_item(self, item):
        self.subitems[item.uid] = item
        
    def iter_all(self):
        for it in self.subitems.values():
            yield from it.iter_all()
            yield it
        
        
class GanttItem(GanttBase):
    def __init__(self, project, name, duration):
        super().__init__(name)
        self.duration = duration
        self.uid = project.allocate_uid()
        self.project = project
        self.descendents = weakref.WeakSet()
        self.precedents = weakref.WeakSet()
        
    def minstart(self):
        m = 0
        for p in self.precedents:
            mt = p.duration + p.minstart()
            if mt > m:
                m = mt
        return m
        
    def add_precedent(self, it):
        if isinstance(it, int):
            it = self.project.get_by_uid(it)
        self.precedents.add(it)
        
    def add_descendent(self, it):
        if isinstance(it, int):
            it = self.project.get_by_uid(it)
        self.descendents.add(it)
        
    def new_item(self, project, name, duration):
        new = self.__class__(project, name, duration)
        self.add_descendent(new)
        new.add_precedent(self)
        self.add_item(new)
        
    def iter_items(self):
        return iter(self.subitems.values())
        
    def free(self):
        for it in self.iter_items():
            it.free()
        self.subitems = []
        self.project = None    
        
    def __repr__(self):
        return "GanttItem(%r, %r, %r)"%(self.project, self.name, self.duration)
        
        
class GanttProject(GanttBase):
    def __init__(self, name):
        super().__init__(name)
        self._cur_id = 0
        
    def __repr__(self):
        return "GanttProject(%r)"%self.name
        
    def allocate_uid(self):
        rv = self._cur_id
        self._cur_id += 1
        return rv
    
    def get_by_uid(self, n):
        return self.subitems[n]
    
    def new_item(self, name, duration, prec=None):
        new = GanttItem(self, name, duration)
        if prec is not None:
            if isinstance(prec, int):
                prec = self.get_by_uid(prec)
            new.add_precedent(prec)
            prec.add_descendent(new)
        self.add_item(new)
        return new
    
    def print(self, fp=None):
        if fp is None:
            fp = sys.stdout
        
        tmpl = "%-40s |%s"
        
        out = []
        for it in self.iter_all():
            out.append((it.name, it.minstart(), it.duration))
            
        out.sort(key=lambda t: t[1])
        last = out[-1]
        end = last[1] + last[2]
        end = math.ceil(end/60)*60 
        
        nhours = math.ceil(end / 60)
        xaxis = []
#         for i in range(nhours):
#             s = str(i)
#             sl = len(s)
#             nleft = int((12 // 2) - (sl // 2))
#             nright = 12 - nleft - sl
#             xaxis.append("%s%s%s"%(nleft * " ", s, nright * " "))

            
        nhours = math.ceil(end / 60)
        nright = 11
        for i in range(nhours):
            s = str(i)
            sl = len(s)
            xaxis.append(" " * nright)
            xaxis.append(s)
            nright = int(10 - (sl // 2))
            
            
        sout = tmpl%("Activity", " ".join(xaxis))
        print(sout, file=fp)
        print("_"*len(sout), file=fp)
        
        for name, start, dur in out:
            sb = []
            i = 0
            dend = start + dur
            
            for j in range(int(end/5)+1):
                
                if i > 0 and not i % 60:
                    sb.append("|")
                if start <= (5*j) < dend:
                    sb.append("-")
                else:
                    sb.append(" ")
                i += 5
                
            padbar = "".join(sb)
            print(tmpl%(name, padbar), file=fp)
            

In [135]:
def add_to_proj(proj, qp, last=None):
    for name, dur in qp:
        new = proj.new_item(name, dur, last)
        last = new
    return last

proj = GanttProject("SemmaIQOQ")
last = add_to_proj(proj, iqp)
add_to_proj(proj, oqp, last)
with open("test.txt", 'w') as f:
    proj.print(f)

In [49]:
proj = GanttProject("SemmaIQOQ")
it = proj.new_item("Document Approvals", 10)
proj.new_item("Test Equipment", 10, it)
proj.print()

Document Approvals                       | --
Test Equipment                           |   --


In [31]:
from officelib.xllib import *
xl = Excel()

In [50]:
wb = xl.Workbooks("IOQ Time Estimate.xlsx")
ws = wb.Worksheets("IOQ")
cells = ws.Cells
cr = cells.Range

def data(c1):
    c2 = c1.End(xlc.xlDown).Offset(1, 6)
    return cr(c1, c2).Value

iq1 = cr("A3")
iq = data(iq1)

oq1 = cr("A27")
oq = data(oq1)

In [51]:
def parse(data):
    out = []
    for name, _, _, _, _, total in data:
        out.append((name, total / 60))
    return out

iqp = parse(iq)
oqp = parse(oq)

In [105]:
def add_to_proj(proj, qp, last=None):
    for name, dur in qp:
        new = proj.new_item(name, dur, last)
        last = new
    return last

proj = GanttProject("SemmaIQOQ")
last = add_to_proj(proj, iqp)
add_to_proj(proj, oqp, last)
with open("test.txt", 'w') as f:
    proj.print(f)

In [467]:
tasks = [
    # Do all approvals at the same time
    ('IQDocument Approvals', 10.0, False, 1),
    ('OQDocument Approvals', 10.0, False, 1),
    
    # Same with equipment
    ('IQTest Equipment', 10.0, False, 1),
    ('OQTest Equipment', 30.0, False, 1),
    
    ('System Accessories', 10.0, False, 0),
    ('Product Info', 10.0, False, 0),
    ('Physical inspection', 20.0, False, 0),
    ('Plant utilities', 10.0, False, 1),
    ('Gas Lines', 20.0, False, 1),
    ('Bios Config / battery', 10.0, False, 0),
    ('Power Up', 10.0, False, 0),
    ('Software verif', 5.0, False, 0),
    ('OS Config', 5.0, False, 0),
    ('McAfee Config', 5.0, False, 0),
    ('Inbound Firewall', 10.0, False, 0),
    ('Outbound Firewall', 10.0, False, 0),
    ('System Vars', 10.0, False, 0),
    ('Alarms Off', 10.0, False, 0),
    ('Alarms On', 10.0, False, 0),
    ('Logging Off', 10.0, False, 0),
    ('Logging On', 10.0, False, 0),
    ('Email Settings', 5.0, False, 0),
    ('Verify User Groups', 5.0, False, 0),
    ('IQProtocol Completion', 10.0, False, 2),
    
    # OQ
    ('Power On', 10.0, False, 0),
    ('Vessel Fit', 10.0, False, 0),
    ('Level Verification', 10.0, False, 0),
    ('Agitation Control', 20.0, False, 0),
    ('Transfer Config Files', 20.0, False, 1),
    ('Temp Setup', 5, False, 0),
    ('Temp Wait 1', 390.0, True, 0),
    ('Temp Check', 25, False, 0),
    ('Temp Wait 2', 240, True, 0),
    ('Temp Check 2', 25, False, 0),
    ('pH / DO Sensor Input', 5.0, False, 0),
    ('pH Setup', 10, False, 0),
    ('pH Wait 1', 60, True, 0),
    ('pH Check', 5, False, 0),
    ('DO Setup', 10, False, 0),
    ('DO Wait 1', 40, True, 0),
    ('DO Check 1', 5, False, 0),
    ('DO Wait 2', 40, True, 0),
    ('DO Check 2', 5, False, 0),
    ('Gas Request', 5.0, False, 0),
    ('Gas Flow Accuracy ', 15.0, False, 0),
    ('Pumps', 5.0, False, 0),
    ('Filter Oven Setup', 5, False, 0),
    ('Filter Oven Wait', 60, True, 0),
    ('Filter Oven Check', 5, False, 0),
    ('Recipe Verification', 5.0, False, 0),
    ('Power Loss & Restore', 15.0, False, 0),
    ('Interlocks', 10.0, False, 0),
    ('Process Alarms', 10.0, False, 0),
    ('Clear Alarms', 5.0, False, 0),
    ('Cleanup', 10.0, False, 0),
    ('Flash Drive Verif', 10.0, False, 1),
    ('OQProtocol Completion', 10.0, False, 2)
]
seen = set()
#nt = []
for a,b,c,d in tasks:
    if a in seen:
        print("DUP: %r"%a)
    seen.add(a)
    #nt.append((a,b,c,int(d)))
#clipboard.copy(repr(nt).replace(", (", ",\n    ("))

In [472]:
import datetime

def fmt(dt):
    return dt.strftime("%I:%M %p")

def fmt2(dt):
    return dt.strftime("%m/%d/%y %I:%M:%S %p")

current = start

class Task():
    def __init__(self, br, name, dur, para, prec, once_only):
        self.br = br
        self.name = name
        self.dur = dur
        self.st = None
        self.prec = prec
        self.para = para
        self._done = False
        if once_only:
            once_only = 2
        self.once_only = once_only
        self.once_seen = False
        
    def running(self):
        return self.st is not None
    
    def finish(self):
        self._done = True
        
    def start(self, dt):
        self.st = dt
        
    def done(self, dt):
        if self._done:
            return True
        if not self.st:
            return False
        elapsed = self.elapsed(dt)
        return elapsed >= self.dur
        
    def elapsed(self, dt):
        return (dt - self.st).total_seconds() / 60 
    
    def __repr__(self):
        return "Task(%r, %r, Task<%s>)"%(self.name, self.dur, self.prec.name if self.prec else "Empty")
    

ioq_tasks = []

for br in range(4):
    br += 1
    prev = None
    for name, dur, para, once_only in tasks:
        task = Task(br, name, dur, para, prev, once_only)
        prev = task
        ioq_tasks.append(task)

start = startstart = current = datetime.datetime(2018, 12, 6, 9)
end_of_day = datetime.datetime(2018, 12, 6, 12+6)
oneday = datetime.timedelta(hours=24)
minute = datetime.timedelta(minutes=5)
brs = [
    0,
    0,
    0,
    0,
    0
]

import collections
once_done = collections.defaultdict(lambda: 0)

out = []
def showtask(task, start, end):
    if task.once_only:
        br = 0
    else:
        br = task.br
    print("%d %-30s %30s %30s %s"%(br, task.name, fmt(start), fmt(end), "para" if task.para else ""))
    out.append((br, "<%d>%s"%(br, task.name), task.dur / 1440, fmt2(start)))

def elapsed():
    return (current - start).total_seconds() / 60

while ioq_tasks:
    #el = elapsed()
    
    for task in ioq_tasks:
        #import pdb;pdb.set_trace()
#         if brs == [0, 1,1,1, 1]:
#             import pdb; pdb.set_trace()
        if task.para and task.running():
            if task.done(current):
                ioq_tasks.remove(task)
                brs[task.br] = 0
                task.finish()
                # free all bioreactors availability
                if task.once_only == 2:
                    print("Freeing bioreactors")
                    for i in range(len(brs)):
                        brs[i] = 0
                break
            else:
                continue
                
        if (task.prec is None or task.prec.done(current)):
            if brs[task.br] == 0:
                
                parallel = task.para
                not_end_of_day = datetime.timedelta(minutes=task.dur) + current < end_of_day
                
                if not_end_of_day or parallel:
                    if task.once_only == 1:
                        if task.name in once_done:
                            task.finish()
                            ioq_tasks.remove(task)
                            break
                        else:
                            once_done[task.name] += 1  # fallthrough to run
                    elif task.once_only == 2:
                        once_done[task.name] += 1
                        if once_done[task.name] > 4:  # sanity check
                            raise ValueError("%s seen more than 4 times"%task.name)
                        if once_done[task.name] < 4:  # wait until the number of tasks reaches 4
                            task.finish()
                            ioq_tasks.remove(task)
                            brs[task.br] = 1
                            break
                        
                
                if parallel:
                    task.start(current)
                    end = current + datetime.timedelta(minutes=task.dur)
                    showtask(task, current, end)
                    brs[task.br] = 1
                    
                elif not_end_of_day:
                    task.start(current)
                    end = current + datetime.timedelta(minutes=task.dur)
                    ioq_tasks.remove(task)
                    showtask(task, current, end)
                    task.finish()
                    current = end
                    if task.once_only == 2:
                        for i in range(len(brs)):
                            brs[i] = 0
                    else:
                        brs[task.br] = 0
                    break
    else:
        current += minute
        if current > end_of_day:
            start += oneday
            end_of_day += oneday
            current = start

                


#print("done")

0 IQDocument Approvals                                 09:00 AM                       09:10 AM 
0 OQDocument Approvals                                 09:10 AM                       09:20 AM 
0 IQTest Equipment                                     09:20 AM                       09:30 AM 
0 OQTest Equipment                                     09:30 AM                       10:00 AM 
1 System Accessories                                   10:00 AM                       10:10 AM 
1 Product Info                                         10:10 AM                       10:20 AM 
1 Physical inspection                                  10:20 AM                       10:40 AM 
2 System Accessories                                   10:40 AM                       10:50 AM 
2 Product Info                                         10:50 AM                       11:00 AM 
2 Physical inspection                                  11:00 AM                       11:20 AM 
3 System Accessories                    

In [478]:
ws = wb.Worksheets("Gantt3")
cells = ws.Cells
cr = cells.Range

def paste(c1, values):
    c2 = c1.Offset(len(values), len(values[0]))
    cr(c1, c2).Value = values

def clear(c1):
    if c1.Value and c1.Offset(2,1).Value:
        c2 = c1.End(xlc.xlDown).End(xlc.xlToRight)
        cr(c1, c2).Clear()
        
def copy(c1):
    c2 = c1.Offset(len(values), len(values[0]))
    return cr(c1, c2).Value
    
c1 = cr("B3")
clear(c1)
paste(c1, out)
values = copy(c1)  # im too lazy to convert str to datetime, so i let xl do it

In [479]:
def RGB(red, green, blue):
    return (red << 16) | (green << 8) | blue

colors = {
    '0':RGB(0,0,0),
    '1':RGB(255,0,0),
    '2':RGB(0,255,0),
    '3':RGB(0,0,255),
    '4':RGB(120,120,0),
}

def scale_chart(chart):
    series = chart.SeriesCollection("Duration")
    series2 = chart.SeriesCollection("Start Time")
    values = series2.Values
    y1 = values[0]
    yn = values[-1]
    yax  = chart.Axes(xlc.xlValue)
    yax.MinimumScale = y1
    yax.MaximumScale = yn + series.Values[-1] + 1/24
    
def format_barchart(chart):
    with screen_lock(xl):
        series = chart.SeriesCollection("Duration")
        points = list(series.Points())
        xvalues = series.XValues
        for x, p in zip(xvalues, points):
            p.Interior.Color = colors[x[1]]
        scale_chart(chart)

In [480]:
import pytz

class Task2():
    def __init__(self, br, name, start):
        self.br = br
        self.name = name
        self.start = start
        self.end = 0
        self.last_ts = None
    def __repr__(self):
        return "Task2(%r, %r, %r)" % (self.name, self.start, self.end)
    
    def tuple(self):
        return self.br, self.name, datetime.datetime.fromtimestamp(self.start, pytz.utc), (self.end - self.start) / 86400


sets = []
last_br = 0
last_ts = 0
last_name = 0
last_day = values[0][-1].day
curr = None
last_dur = 0

def append_task(name, start, task):
    task.end = start
    #print(task.name, name)
    if task.name != name:
        task.name += " - " + name
    sets.append(task.tuple())
    
    
for br, act, dur, start in values:
    sts = start.timestamp()
    if curr is None:
        curr = Task2(br, act, sts)
    else:
        if last_day != start.day:
            append_task(last_name, last_ts + last_dur * 3600, curr)
            curr = Task2(br, act, sts)
            
        elif br != last_br:
            append_task(last_name, sts, curr)
            curr = Task2(br, act, sts)
    
    last_br = br
    last_ts = sts
    last_day = start.day
    last_name = act
    last_dur = dur
    
append_task(last_name, last_ts + dur * 3600, curr)

In [482]:
c1 = cr("G3")
clear(c1)
paste(c1, sets)

In [506]:
def update_chart_data(sets):
    day = sets[0][2].day  # first day
    nch = 2
    istart = 1
    chart = ws.ChartObjects(nch).Chart
    count = ws.ChartObjects().Count
    c1 = cr("H3")

    isets = iter(sets)
    i = 0

    def setvalues():
        chart.HasTitle = True
        chart.ChartTitle.Text = "Day %d"%(nch-1)
        xr = cr(c1, c1.Offset(i - istart, 1))
        y1r = cr(c1.Offset(1, 2), c1.Offset(i - istart, 2))
        y2r = cr(c1.Offset(1, 3), c1.Offset(i - istart, 3))
        print(xr.GetAddress(), y1r.GetAddress(), y2r.GetAddress())
        s1 = chart.SeriesCollection("Start Time")
        s2 = chart.SeriesCollection("Duration")
        s1.XValues = xr
        s1.Values = y1r
        s2.XValues = xr
        s2.Values = y2r

    while True:
        i += 1
        val = next(isets, None)
        if val is None:
            setvalues()
            break
        else:
            br, task, start_time, duration = val
        if start_time.day != day:
            setvalues()
            day = start_time.day
            c1 = c1.Offset(i - istart + 1, 1)
            nch += 1
            istart = i
            if nch > count:
                raise ValueError("Not enough charts for all days")
            chart = ws.ChartObjects(nch).Chart
            print()
            print("NEW CHART: %d"%nch)
        print("Task: ", task)
with screen_lock(xl):
    update_chart_data(sets)

Task:  <0>IQDocument Approvals - <0>OQTest Equipment
Task:  <1>System Accessories - <1>Physical inspection
Task:  <2>System Accessories - <2>Physical inspection
Task:  <3>System Accessories - <3>Physical inspection
Task:  <4>System Accessories - <4>Physical inspection
Task:  <0>Plant utilities - <0>Gas Lines
Task:  <1>Bios Config / battery - <1>Verify User Groups
Task:  <2>Bios Config / battery - <2>Verify User Groups
Task:  <3>Bios Config / battery - <3>Outbound Firewall
$H$3:$H$11 $I$3:$I$11 $J$3:$J$11

NEW CHART: 3
Task:  <3>System Vars - <3>Verify User Groups
Task:  <4>Bios Config / battery - <4>Verify User Groups
Task:  <0>IQProtocol Completion
Task:  <1>Power On - <1>Agitation Control
Task:  <2>Power On - <2>Agitation Control
Task:  <3>Power On - <3>Agitation Control
Task:  <4>Power On - <4>Agitation Control
Task:  <0>Transfer Config Files
Task:  <1>Temp Setup - <1>Temp Wait 1
Task:  <2>Temp Setup - <2>Temp Wait 1
Task:  <3>Temp Setup - <3>Temp Wait 1
Task:  <4>Temp Setup - <4>Te

In [486]:
with screen_lock(xl):
    for i, co in enumerate(ws.ChartObjects(), 1):
        chart = co.Chart
        format_barchart(chart)
        if i != 1:
            co.Width = 610
            co.Height = 328

In [450]:
scale_chart(xl.Selection.Parent)
format_barchart(xl.Selection.Parent)
p = xl.Selection.Parent.Parent
p.Height, p.Width = h, w

In [437]:
def xax(c):
    return c.Axes(xlc.xlValue)

xax(xl.Selection.Parent).NumberFormat = xax(chart1).NumberFormat

AttributeError: '<win32com.gen_py.Microsoft Excel 12.0 Object Library.Axis instance at 0x1789501665008>' object has no attribute 'NumberFormat'

In [447]:
p = xl.Selection.Parent.Parent
p.Height, p.Width = h, w

In [449]:
p = xl.Selection.Parent.Parent
h, w = p.Height, p.Width

In [484]:
h, w

(327.9270935058594, 609.3818359375)